In [4]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client google-cloud-language

   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.8 MB 3.3 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/12.8 MB 3.7 MB/s eta 0:00:04
   ------- -------------------------------- 2.4/12.8 MB 3.8 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/12.8 MB 3.8 MB/s eta 0:00:03
   ---------- ----------------------------- 3.4/12.8 MB 3.2 MB/s eta 0:00:03
   ------------- -------------------------- 4.2/12.8 MB 3.6 MB/s eta 0:00:03
   ---------------- ----------------------- 5.2/12.8 MB 3.5 MB/s eta 0:00:03
   ------------------- -------------------- 6.3/12.8 MB 3.8 MB/s eta 0:00:02
   ---------------------- ----------------- 7.1/12.8 MB 3.8 MB/s eta 0:00:02
   ------------------------ --------------- 7.9/12.8 MB 3.8 MB/s eta 0:00:02
   --------------------------- ------------ 8.7/12.8 MB 3.8 MB/s eta 0:00:02
   ----------------------------- ---------- 9.4/12.8 MB 3.8 MB/s eta 0:00:01
   ---

In [1]:
#all the necessary imports
from datetime import datetime
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os

In [2]:
# import the google_API_Key for genai usage
# import the client secret
from constants import google_API_key, client_secret_path

In [3]:
# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly',
          'https://www.googleapis.com/auth/gmail.send',
          'https://www.googleapis.com/auth/gmail.modify']

def authenticate():
    creds = None
    token_file = 'token.json'

    # Delete token.json if it's invalid
    if os.path.exists(token_file):
        try:
            creds = Credentials.from_authorized_user_file(token_file, SCOPES)
        except ValueError:
            os.remove(token_file)

    # If there are no valid credentials, authenticate with credentials.json
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                client_secret_path, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the new token
        with open(token_file, 'w') as token:
            token.write(creds.to_json())

    return creds

In [4]:
creds = authenticate()
print("Authentication successful!")

Authentication successful!


In [5]:
from googleapiclient.discovery import build
from base64 import urlsafe_b64decode

def list_emails(creds):
    service = build('gmail', 'v1', credentials=creds)

    # List messages with a query filter to get only emails in the inbox sent to you
    results = service.users().messages().list(userId='me', maxResults=10, q='in:inbox to:me').execute()
    messages = results.get('messages', [])

    emails = []

    if not messages:
        return emails
    else:
        for message in messages:
            # Get message details
            msg = service.users().messages().get(userId='me', id=message['id'], format='full').execute()

            # Extract message data
            email_id = message['id']
            payload = msg.get('payload', {})
            headers = payload.get('headers', [])
            
            # Extract subject
            subject = next((header['value'] for header in headers if header['name'] == 'Subject'), "No Subject")

            # Extract date
            date = next((header['value'] for header in headers if header['name'] == 'Date'), "No Date")
            
            # Extract message body (supports plain text only)
            body_data = None
            if payload.get('body', {}).get('data'):
                body_data = payload['body']['data']
            elif payload.get('parts'):
                for part in payload['parts']:
                    if part.get('mimeType') == 'text/plain' and part.get('body', {}).get('data'):
                        body_data = part['body']['data']
                        break
            
            # Decode the message body
            body = urlsafe_b64decode(body_data).decode('utf-8') if body_data else "No Message Body"

            # Append email details to the list
            emails.append({
                'email_id': email_id,
                'date': date,
                'subject': subject,
                'body': body
            })
    
    return emails

In [6]:
# emails = list_emails(creds)
# for email in emails:
#     print(email)

In [7]:
from email.mime.text import MIMEText
import base64

def send_email(to, subject, message_text):
    creds = authenticate()
    service = build('gmail', 'v1', credentials=creds)

    # Create the email message
    message = MIMEText(message_text)
    message['to'] = to
    message['subject'] = subject
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

    # Send the email
    message = {'raw': raw}
    send_message = service.users().messages().send(userId='me', body=message).execute()
    print(f"Message sent: {send_message['id']}")

In [8]:
# send_email("ammaryasser.eng@gmail.com", "Hello again", "this is a test message")

In [9]:
def forward_email(to, original_email, emailType, summary, recommendation):
    """
    Forward an email to another recipient.
    :param to: Recipient's email address.
    :param original_email: Dictionary containing the original email's details (subject, body).
    """
    creds = authenticate()
    service = build('gmail', 'v1', credentials=creds)

    # Prepare the forwarded email content
    subject = f"Fwd: [AI-Agent] Forwarded Email: {emailType} - {original_email['subject']}"  # Prefix the subject with "Fwd:"

    body = f"""\
AI Summary:
Type: {emailType}
------------------------------
Content Overview: {summary}
------------------------------
Recommended Action: {recommendation}
------------------------------
Original Email:
{original_email['body']}
"""

    # Create the email message
    message = MIMEText(body)
    message['to'] = to
    message['subject'] = subject
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

    # Send the email
    message = {'raw': raw}
    send_message = service.users().messages().send(userId='me', body=message).execute()
    print(f"Message forwarded: {send_message['id']}")

# Example usage

# msg = generateQuery(emails[0])
# response = model.generate_content(msg)
# category, summary, recommendation = extract_category_and_summary(response.text)
# forward_email("ammar123yasserals@gmail.com", emails[0], category, summary, recommendation)


In [10]:
import google.generativeai as genai

genai.configure(api_key=google_API_key)
model = genai.GenerativeModel("gemini-1.5-flash")
# response = model.generate_content("Explain how AI works")
# print(response.text)

In [11]:
# generate the iniquiry for asking google gemini
def generateQuery(email):
    message = "email subject : " + email['subject'] + ", email body: " + email['body'] + "\n I want to categorize this email into either Sales, Partnership, Personal or Inquiry \n and I want to get a summary for that email \n I want the response in the format Category: <category in one word> \n Summary: <summary> \n Recommendation: <Recommended Action> \n if its meant as a job recommendation from linked in categorize it as Personal"
    return message

In [12]:
# emails[1]

In [13]:
# msg = generateQuery(emails[1])
# print(msg)

In [14]:
# response = model.generate_content(msg)


In [15]:
# print(response.text)

In [16]:
# exracts the category, summary, and the recommendation from the AI response 
def extract_category_and_summary(msg):
    data = msg.split("Summary: ")
    summary_recommendation = data[1].split("Recommendation: ")
    summary = summary_recommendation[0]
    recommendation = summary_recommendation[1]
    category = data[0].split("Category: ")[1]
    return category, summary, recommendation

In [17]:
# # Get results
# category, summary, recommendation = extract_category_and_summary(response.text)
# print(category)
# print(summary)
# print(recommendation)

In [18]:
# a dict to save processed emails
email_id_status = {}

In [19]:
contacts = [
    {"name": "John Doe", "title": "CEO", "email": "ammar123yasserals@gmail.com", "manages": "Partnership"},
    {"name": "Jane Smith", "title": "CTO", "email": "ammar123yasserals@gmail.com", "manages": "Inquiry"},
    {"name": "Edard Stark", "title": "Sales Manager", "email": "ammarelsayed.game@gmail.com", "manages": "Sales"},
    {"name": "Tywin lannister", "title": "me", "email": "ammarelsayed.game@gmail.com", "manages": "Personal"},
]

In [ ]:
from datetime import datetime, timezone

# Get the current time in UTC timezone
currentTime = datetime.now(timezone.utc)

# Main loop to continuously check and process emails
while True:
    # Print the start of a new loop iteration with the current time
    print("new loop at ", datetime.now())
    
    # Retrieve the list of emails using the provided credentials
    emails = list_emails(creds)
    
    # Iterate through each email in the list
    for email in emails:
        # Check if the email has not been processed yet
        if(email_id_status.get(email["email_id"], False) == False):
            # Mark the email as processed
            email_id_status[email["email_id"]] = True
            
            try:
                # Define the date format used in the email's date field
                date_format = '%a, %d %b %Y %H:%M:%S %z'
                
                # Parse the email's date string into a datetime object
                parsed_date = datetime.strptime(email['date'], date_format)
                
                # Convert the parsed date to UTC timezone
                parsed_date = parsed_date.astimezone(timezone.utc)
                
                # Check if the email's date is after the current time
                if(parsed_date > currentTime):
                    # Process the email
                    print("process email")
                    print("email: ", email)
                    
                    # Mark the email as processed
                    email_id_status[email["email_id"]] = True
                    
                    # Generate a query based on the email content
                    query = generateQuery(email)
                    
                    # Get a response from the AI model based on the query
                    AI_response = model.generate_content(query)
                    
                    # Extract category, summary, and recommendation from the AI response
                    category, summary, recommendation = extract_category_and_summary(AI_response.text)
                    
                    # Print the category of the email
                    print("category: ", category)
                    
                    # Forward the email based on its category
                    if("Partnership" in category):
                        forward_email(contacts[0]["email"], email, category, summary, recommendation)
                    elif("Inquiry" in category):
                        forward_email(contacts[1]["email"], email, category, summary, recommendation)
                    elif("Sales" in category):
                        forward_email(contacts[2]["email"], email, category, summary, recommendation)
                    elif("Personal" in category):
                        forward_email(contacts[3]["email"], email, category, summary, recommendation)
                    else:    
                        # Handle unexpected categories
                        print("ERROR! summary outside summary values, category = ", category)
                    
                    # Print a separator for readability
                    print("-" * 50)
            except Exception as e:
                # Handle any exceptions that occur during processing
                print("An error occurred:", str(e))

new loop at  2025-01-28 09:49:57.876269
An error occurred: unconverted data remains:  (UTC)
An error occurred: unconverted data remains:  (UTC)
new loop at  2025-01-28 09:50:00.754720
new loop at  2025-01-28 09:50:03.522812
new loop at  2025-01-28 09:50:06.248814
new loop at  2025-01-28 09:50:09.051782
new loop at  2025-01-28 09:50:11.823628
new loop at  2025-01-28 09:50:14.645615
new loop at  2025-01-28 09:50:17.358799
new loop at  2025-01-28 09:50:20.067658
new loop at  2025-01-28 09:50:22.746566
new loop at  2025-01-28 09:50:25.518118
new loop at  2025-01-28 09:50:28.293546
new loop at  2025-01-28 09:50:30.950425
new loop at  2025-01-28 09:50:33.668900
new loop at  2025-01-28 09:50:36.421886
new loop at  2025-01-28 09:50:39.093616
new loop at  2025-01-28 09:50:42.189039
new loop at  2025-01-28 09:50:44.930267
new loop at  2025-01-28 09:50:47.623440
new loop at  2025-01-28 09:50:50.279178
new loop at  2025-01-28 09:50:53.019966
new loop at  2025-01-28 09:50:56.248062
new loop at  202

KeyboardInterrupt: 